In [57]:
! pip install deap


In [58]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

In [59]:
# Definição do problema: Bin Packing
N_ITENS = 50  # Número de itens
CAPACIDADE_BIN = 100  # Capacidade máxima de cada caixa

# Gerar pesos dos itens aleatórios (entre 10 e 50)
np.random.seed(42)
pesos_itens = np.random.randint(10, 50, size=N_ITENS)
print(f"Pesos dos itens: {pesos_itens}")
print(f"Capacidade de cada bin: {CAPACIDADE_BIN}")

Pesos dos itens: [48 38 24 17 30 48 28 32 20 20 33 45 49 33 12 31 11 33 39 47 11 30 42 21
 31 34 36 37 25 24 12 46 16 30 18 48 27 13 34 23 18 35 11 29 37 16 17 44
 23 26]
Capacidade de cada bin: 100


In [60]:
def avaliar(individuo):
    """Calcula o número de bins usados."""
    bins = {}  # Dicionário para armazenar os bins e seus pesos
    for i, bin_index in enumerate(individuo):
        if bin_index not in bins:
            bins[bin_index] = 0
        bins[bin_index] += pesos_itens[i]

    # Penalizar bins que excedem a capacidade
    num_bins = len(bins)
    for peso in bins.values():
        if peso > CAPACIDADE_BIN:
            num_bins += 10  # Penalidade alta para bins inválidos

    return num_bins,

In [61]:
# Criar classes para fitness e indivíduo
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizar número de bins
creator.create("Individual", list, fitness=creator.FitnessMin)

# Inicializar a toolbox DEAP
toolbox = base.Toolbox()
toolbox.register("genes", random.randint, 0, N_ITENS)  # Genes representam o bin de cada item
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.genes, n=N_ITENS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", avaliar)
toolbox.register("mate", tools.cxTwoPoint)  # Cruzamento de dois pontos
toolbox.register("mutate", tools.mutUniformInt, low=0, up=N_ITENS, indpb=0.1)  # Mutação
toolbox.register("select", tools.selTournament, tournsize=3)  # Seleção por torneio

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [62]:
def busca_local(individuo):
    """Busca local: remove e reinsire um item aleatoriamente."""
    for i in range(N_ITENS):
        # Escolhe um item aleatoriamente
        item_index = random.randint(0, N_ITENS - 1)

        # Tenta movê-lo para um bin diferente
        novo_bin = random.randint(0, N_ITENS)

        # Cria um novo indivíduo com a mudança
        novo_individuo = individuo[:]
        novo_individuo[item_index] = novo_bin

        # Se a nova solução for melhor, substitui a antiga
        if avaliar(novo_individuo) < avaliar(individuo):
            individuo[:] = novo_individuo[:]
            break  # Sai do loop se encontrar uma solução melhor

    return individuo,

In [63]:
toolbox.register("busca_local", busca_local)




In [64]:
def algoritmo_memetico(pop_size=100, ngen=100, p_cx=0.7, p_mut=0.2, p_local=0.3):
    """Executa um Algoritmo Memético com GA + Busca Local."""
    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)

    for gen in range(ngen):
        # Seleção e reprodução
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < p_cx:
                toolbox.mate(child1, child2)
                del child1.fitness.values, child2.fitness.values

        for mutant in offspring:
            if random.random() < p_mut:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Reavaliação dos filhos
        for ind in offspring:
            if not ind.fitness.valid:
                ind.fitness.values = toolbox.evaluate(ind)

        # Aplicação da busca local a uma fração da população
        for ind in random.sample(offspring, int(p_local * len(offspring))):
           toolbox.busca_local(ind)
           ind.fitness.values = toolbox.evaluate(ind)

        pop[:] = offspring
        hof.update(pop)

        # Coleta estatísticas
        record = stats.compile(pop)
        print(f"Geração {gen}: Min = {record['min']}, Avg = {record['avg']}")

    return pop, hof

In [65]:
resultados = []
for i in range(10):
    pop, hof = algoritmo_memetico(pop_size=100, ngen=100, p_cx=0.7, p_mut=0.2, p_local=0.3)
    resultados.append(avaliar(hof[0])[0])  # Armazenar o número de bins


media_resultados = np.mean(resultados)
print("Média do número de bins:", media_resultados)
# Melhor solução encontrada
print("Melhor solução:", hof[0])
print("Valor total:", avaliar(hof[0])[0])
peso_total = 0
for bin_index in range(N_ITENS + 1): # Considerando todos os bins possíveis
    peso_bin = sum(pesos_itens[i] for i in range(N_ITENS) if hof[0][i] == bin_index)
    peso_total += peso_bin

print("Peso total:", peso_total)
print("tamanho da Solução:",len(hof[0]))

Geração 0: Min = 29.0, Avg = 43.73
Geração 1: Min = 28.0, Avg = 40.55
Geração 2: Min = 29.0, Avg = 37.73
Geração 3: Min = 28.0, Avg = 37.89
Geração 4: Min = 27.0, Avg = 37.3
Geração 5: Min = 25.0, Avg = 35.15
Geração 6: Min = 27.0, Avg = 35.39
Geração 7: Min = 27.0, Avg = 35.73
Geração 8: Min = 25.0, Avg = 35.68
Geração 9: Min = 24.0, Avg = 33.75
Geração 10: Min = 23.0, Avg = 34.52
Geração 11: Min = 23.0, Avg = 36.05
Geração 12: Min = 23.0, Avg = 34.93
Geração 13: Min = 23.0, Avg = 32.67
Geração 14: Min = 22.0, Avg = 31.79
Geração 15: Min = 22.0, Avg = 30.95
Geração 16: Min = 22.0, Avg = 29.26
Geração 17: Min = 21.0, Avg = 26.67
Geração 18: Min = 20.0, Avg = 24.68
Geração 19: Min = 20.0, Avg = 24.06
Geração 20: Min = 20.0, Avg = 23.87
Geração 21: Min = 20.0, Avg = 22.61
Geração 22: Min = 19.0, Avg = 22.8
Geração 23: Min = 20.0, Avg = 23.66
Geração 24: Min = 20.0, Avg = 22.61
Geração 25: Min = 20.0, Avg = 21.95
Geração 26: Min = 19.0, Avg = 22.35
Geração 27: Min = 19.0, Avg = 21.6
Geraç

In [66]:
def fusao_bins(individuo):
    """Busca local: Fusão de Bins."""
    bins = {}  # Dicionário para armazenar os bins e seus pesos
    for i, bin_index in enumerate(individuo):
        if bin_index not in bins:
            bins[bin_index] = 0
        bins[bin_index] += pesos_itens[i]

    # Tentativa de fusão de bins
    bins_ids = list(bins.keys())
    for i in range(len(bins_ids)):
        for j in range(i + 1, len(bins_ids)):
            bin1_id = bins_ids[i]
            bin2_id = bins_ids[j]

            # Verificar se a fusão é possível
            if bins[bin1_id] + bins[bin2_id] <= CAPACIDADE_BIN:
                # Fundir os bins
                for k in range(len(individuo)):
                    if individuo[k] == bin2_id:
                        individuo[k] = bin1_id

                # Atualizar o dicionário de bins
                bins[bin1_id] += bins[bin2_id]
                del bins[bin2_id]

                # Retornar o indivíduo modificado
                return individuo,

    # Se nenhuma fusão foi possível, retornar o indivíduo original
    return individuo,

In [67]:
toolbox.register("f", fusao_bins)


In [68]:
def algoritmo_memetico(pop_size=100, ngen=100, p_cx=0.7, p_mut=0.2, p_local=0.3):
    """Executa um Algoritmo Memético com GA + Busca Local."""
    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)

    for gen in range(ngen):
        # Seleção e reprodução
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < p_cx:
                toolbox.mate(child1, child2)
                del child1.fitness.values, child2.fitness.values

        for mutant in offspring:
            if random.random() < p_mut:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Reavaliação dos filhos
        for ind in offspring:
            if not ind.fitness.valid:
                ind.fitness.values = toolbox.evaluate(ind)

        # Aplicação da busca local a uma fração da população
        for ind in random.sample(offspring, int(p_local * len(offspring))):
           toolbox.f(ind)
           ind.fitness.values = toolbox.evaluate(ind)

        pop[:] = offspring
        hof.update(pop)

        # Coleta estatísticas
        record = stats.compile(pop)
        print(f"Geração {gen}: Min = {record['min']}, Avg = {record['avg']}")

    return pop, hof

In [69]:
resultados = []
for i in range(10):
    pop, hof = algoritmo_memetico(pop_size=100, ngen=100, p_cx=0.7, p_mut=0.2, p_local=0.3)
    resultados.append(avaliar(hof[0])[0])  # Armazenar o número de bins


media_resultados = np.mean(resultados)
print("Média do número de bins:", media_resultados)
# Melhor solução encontrada
print("Melhor solução:", hof[0])
print("Valor total:", avaliar(hof[0])[0])
peso_total = 0
for bin_index in range(N_ITENS + 1): # Considerando todos os bins possíveis
    peso_bin = sum(pesos_itens[i] for i in range(N_ITENS) if hof[0][i] == bin_index)
    peso_total += peso_bin

print("Peso total:", peso_total)
print("tamanho da Solução:",len(hof[0]))

Geração 0: Min = 27.0, Avg = 44.19
Geração 1: Min = 27.0, Avg = 40.25
Geração 2: Min = 26.0, Avg = 37.86
Geração 3: Min = 26.0, Avg = 36.26
Geração 4: Min = 25.0, Avg = 36.39
Geração 5: Min = 25.0, Avg = 33.71
Geração 6: Min = 24.0, Avg = 32.76
Geração 7: Min = 23.0, Avg = 29.57
Geração 8: Min = 22.0, Avg = 28.31
Geração 9: Min = 22.0, Avg = 25.96
Geração 10: Min = 21.0, Avg = 25.45
Geração 11: Min = 20.0, Avg = 24.45
Geração 12: Min = 19.0, Avg = 23.74
Geração 13: Min = 18.0, Avg = 23.66
Geração 14: Min = 18.0, Avg = 23.26
Geração 15: Min = 17.0, Avg = 22.26
Geração 16: Min = 17.0, Avg = 21.2
Geração 17: Min = 17.0, Avg = 22.04
Geração 18: Min = 17.0, Avg = 21.57
Geração 19: Min = 17.0, Avg = 22.22
Geração 20: Min = 17.0, Avg = 21.72
Geração 21: Min = 17.0, Avg = 20.65
Geração 22: Min = 17.0, Avg = 21.31
Geração 23: Min = 17.0, Avg = 21.17
Geração 24: Min = 17.0, Avg = 20.63
Geração 25: Min = 17.0, Avg = 19.26
Geração 26: Min = 17.0, Avg = 19.75
Geração 27: Min = 17.0, Avg = 21.0
Gera